In [ ]:

from deepface import DeepFace

import numpy as np
# import pandas as pd
import os

from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# from sklearn.datasets import make_classification
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from qiskit import QuantumCircuit, Aer, transpile
# #from qiskit.providers.aer import AerSimulator
# from qiskit_machine_learning.neural_networks import TwoLayerQNN
# from qiskit_machine_learning.algorithms import NeuralNetworkClassifier
# from qiskit.utils import QuantumInstance
# from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
# #from qiskit.algorithms.optimizers import COBYLA
# from sklearn.metrics import accuracy_score

# only deep face ile face boundary çıkarma from folder Dataset/train
import cv2
def extract_faces_from_folder(folder_path, target_size=(64, 64)):
    X = []
    y = []
    label_map = {}  # Map to convert folder names to numeric labels
    current_label = 0

    for subdir in os.listdir(folder_path):
        subdir_path = os.path.join(folder_path, subdir)
        if os.path.isdir(subdir_path):
            if subdir not in label_map:
                label_map[subdir] = current_label
                current_label += 1
            label = label_map[subdir]

            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)
                try:
                    # Use DeepFace to detect and extract the face
                    face_img = DeepFace.represent(file_path,  enforce_detection=True, model_name="Facenet512", detector_backend='mtcnn')
                    for face in face_img:
                        print(face.get("facial_area"))
                        if face_img is not None:
                            X.append(face.get("embedding"))  # Flatten the image to a 1D array
                            y.append(label)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

    return np.array(X), np.array(y)

def  MinMaxScaler():
    pass

N_SAMPLES = 1800       # Toplam veri örneği sayısı
N_FEATURES_INITIAL = 512 # Klasik ön işleme (CNN) sonrası özellik sayısı (simülasyon)
N_CLASSES = 8          # Sınıf (oda) sayısı
N_QUBITS = 4           # Kuantum devresinde kullanılacak kübit sayısı

SHOTS = 1024           # Ölçüm tekrar sayısı (gürültüyü azaltmak için)
EPOCHS = 30            # Eğitim döngüsü sayısı
LEARNING_RATE = 0.01   # Öğrenme oranı
BATCH_SIZE = 16        # Eğitimde aynı anda işlenecek veri sayısı
#sahte veri oluşturmak yerine face boundary çıkarma ve
# DeepFace kullanarak yüzleri çıkar
# Load the dataset and extract faces
folder_path = 'Dataset/train'
X, y = extract_faces_from_folder(folder_path)



{'x': 518, 'y': 507, 'w': 80, 'h': 92, 'left_eye': (np.int64(576), np.int64(550)), 'right_eye': (np.int64(543), np.int64(547))}
{'x': 385, 'y': 348, 'w': 55, 'h': 81, 'left_eye': (np.int64(428), np.int64(378)), 'right_eye': (np.int64(405), np.int64(382))}
{'x': 227, 'y': 258, 'w': 53, 'h': 70, 'left_eye': (np.int64(265), np.int64(284)), 'right_eye': (np.int64(241), np.int64(286))}
{'x': 152, 'y': 160, 'w': 21, 'h': 31, 'left_eye': (np.int64(168), np.int64(171)), 'right_eye': (np.int64(160), np.int64(174))}
{'x': 427, 'y': 285, 'w': 66, 'h': 92, 'left_eye': (np.int64(487), np.int64(321)), 'right_eye': (np.int64(457), np.int64(323))}
{'x': 275, 'y': 183, 'w': 34, 'h': 45, 'left_eye': (np.int64(300), np.int64(198)), 'right_eye': (np.int64(285), np.int64(200))}
{'x': 316, 'y': 211, 'w': 42, 'h': 58, 'left_eye': (np.int64(346), np.int64(235)), 'right_eye': (np.int64(326), np.int64(235))}
{'x': 305, 'y': 189, 'w': 36, 'h': 49, 'left_eye': (np.int64(332), np.int64(207)), 'right_eye': (np.int6

2025-09-02 14:32:31.562367: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Must provide as many biases as the last dimension of the input tensor: [32] vs. [0,48,48,3]


{'x': 364, 'y': 164, 'w': 45, 'h': 54, 'left_eye': (np.int64(395), np.int64(183)), 'right_eye': (np.int64(375), np.int64(185))}
{'x': 539, 'y': 192, 'w': 55, 'h': 70, 'left_eye': (np.int64(577), np.int64(219)), 'right_eye': (np.int64(553), np.int64(218))}
{'x': 410, 'y': 220, 'w': 70, 'h': 92, 'left_eye': (np.int64(455), np.int64(255)), 'right_eye': (np.int64(423), np.int64(258))}
{'x': 259, 'y': 294, 'w': 90, 'h': 127, 'left_eye': (np.int64(299), np.int64(347)), 'right_eye': (np.int64(264), np.int64(345))}
{'x': 319, 'y': 227, 'w': 67, 'h': 87, 'left_eye': (np.int64(374), np.int64(257)), 'right_eye': (np.int64(344), np.int64(261))}
{'x': 384, 'y': 242, 'w': 81, 'h': 105, 'left_eye': (np.int64(449), np.int64(282)), 'right_eye': (np.int64(413), np.int64(284))}
{'x': 554, 'y': 192, 'w': 56, 'h': 71, 'left_eye': (np.int64(592), np.int64(218)), 'right_eye': (np.int64(567), np.int64(217))}
{'x': 141, 'y': 141, 'w': 22, 'h': 30, 'left_eye': (np.int64(161), np.int64(152)), 'right_eye': (np.in

In [ ]:


print(f"Başlangıç Veri Boyutu (X): {X.shape}")
print(f"Etiket Boyutu (y): {y.shape}")

if X.shape[0] > 0:
    print(f"Extracted {X.shape[0]} faces with {X.shape[1]} features each.")
    print(f"Labels shape: {y.shape}")
else:
    print("No faces were extracted. Check the folder path and image content.")

# --- Adım 2: Özellik Ölçekleme ---
# Özellikleri [0, 1] aralığına ölçekliyoruz.
X = X.reshape(X.shape[0], -1)  # Flatten images if necessary
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print(f"Ölçeklenmiş Veri Boyutu (X_scaled): {X_scaled.shape}")


NameError: name 'X' is not defined